<a href="https://colab.research.google.com/github/purvasingh96/Deep-Reinforcement-Learning/blob/master/4.%20Deep%20Q%20Networks/Credit_Card_Fraud_Detection_via_DQNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading data and Making *gym-fraud-detection* environment

In [0]:
import shutil

shutil.rmtree('/content/gym-fraud-detection')

In [137]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [138]:
import os
print(os.getcwd())

/content


In [140]:
!unzip gym-fraud-detection.zip

Archive:  gym-fraud-detection.zip
   creating: gym-fraud-detection/
  inflating: gym-fraud-detection/setup.py  
  inflating: gym-fraud-detection/README.md  
   creating: gym-fraud-detection/gym_fraud_detection/
  inflating: gym-fraud-detection/gym_fraud_detection/__init__.py  
   creating: gym-fraud-detection/.idea/
  inflating: gym-fraud-detection/.idea/workspace.xml  
  inflating: gym-fraud-detection/.idea/vcs.xml  
  inflating: gym-fraud-detection/.idea/gym-fraud-detection.iml  
  inflating: gym-fraud-detection/.idea/modules.xml  
  inflating: gym-fraud-detection/.idea/encodings.xml  
  inflating: gym-fraud-detection/.idea/misc.xml  
   creating: gym-fraud-detection/.git/
  inflating: gym-fraud-detection/.git/index  
 extracting: gym-fraud-detection/.git/ORIG_HEAD  
  inflating: gym-fraud-detection/.git/FETCH_HEAD  
 extracting: gym-fraud-detection/.git/COMMIT_EDITMSG  
 extracting: gym-fraud-detection/.git/HEAD  
  inflating: gym-fraud-detection/.git/packed-refs  
  inflating: gym-

In [0]:
import os
os.chdir('gym-fraud-detection')

In [142]:
pip install -e .

Obtaining file:///content/gym-fraud-detection
  Found existing installation: gym-fraud-detection 0.0.1
    Can't uninstall 'gym-fraud-detection'. No files were found to uninstall.
  Running setup.py develop for gym-fraud-detection


In [0]:
import gym
import gym_fraud_detection

In [0]:
os.chdir('/content')
env = gym.make('gym-fraud-detection-v0')

In [145]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Discrete(284807)


# Download proper libraries

In [0]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [0]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exploring the custom gym environment

In [148]:
print(env.action_space.n)

2


In [0]:
actions = []
rewards = []
count = 5

while True:
  action = env.action_space.sample()
  reward = env.step(action, 64)
  actions.append(action)
  rewards.append(reward)
  count -= 1
  if count==0:
    break


In [152]:
for i in range(5):
  print("action : ", actions[i], "labelled data : ", env.label_for(i))
  print("reward : ", rewards[i])

action :  1 labelled data :  0.0
reward :  ([], [], [], [])
action :  1 labelled data :  0.0
reward :  ([], [], [], [])
action :  1 labelled data :  0.0
reward :  ([], [], [], [])
action :  0 labelled data :  0.0
reward :  ([], [], [], [])
action :  0 labelled data :  0.0
reward :  ([], [], [], [])


# Exploring data and creating pytorch model

link : https://www.kaggle.com/dakshmiglani/pytorch-credit-card-fraud-prediction-99-8

In [0]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

In [0]:
df = pd.read_csv('./dataset/creditcard.csv')
df.head(1) # give us a sneek preview of the dataset xD

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0


In [0]:
X = df.iloc[:, :-1].values 
y = df.iloc[:, -1].values




sc = StandardScaler()
X = sc.fit_transform(X)

In [183]:
print(df.loc[1])

Time      0.000000
V1        1.191857
V2        0.266151
V3        0.166480
V4        0.448154
V5        0.060018
V6       -0.082361
V7       -0.078803
V8        0.085102
V9       -0.255425
V10      -0.166974
V11       1.612727
V12       1.065235
V13       0.489095
V14      -0.143772
V15       0.635558
V16       0.463917
V17      -0.114805
V18      -0.183361
V19      -0.145783
V20      -0.069083
V21      -0.225775
V22      -0.638672
V23       0.101288
V24      -0.339846
V25       0.167170
V26       0.125895
V27      -0.008983
V28       0.014724
Amount    2.690000
Class     0.000000
Name: 1, dtype: float64


## 2. Defining the Q-Network

In [0]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        self.fc5 = nn.Linear(24, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return x

# Define DQN Agent

In [0]:
import numpy as np
import random 
from collections import namedtuple, deque 

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  #replay buffer size
BATCH_SIZE = 1         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network
EPSILON = 0.8           # probability of chosing on-policy action

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
class Agent():
  def __init__(self, action_size, seed):
    self.action_size = action_size
    self.seed = random.seed(seed)


    # Q - Network
    self.qnet_local = DQN().double().to(device)
    self.qnet_target = DQN().double().to(device)

    self.optimizer = optim.Adam(self.qnet_local.parameters(), lr=0.001)

    self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.add(state, action, reward, next_state, done)


    # learn every 4 timesteps
    self.t_step = (self.t_step+1)%64
    if self.t_step == 0:
      experience = self.memory.sample()
      print('Experience sampled from memory : ', experience)
      self.learn(experience, GAMMA)


  def epsilon_greedy_action(self, state):
    state = state.to(device)
    self.qnet_local.eval()
    with torch.no_grad():
      action_values = self.qnet_local(state).max(1)[1]#.view(1, 1)
    self.qnet_local.train()

    if random.random() < 0.8:
      #print('Predicting action based on QNetwork')
      return action_values
    else:
      #print('Chosing  random actions for the batch')
      random_action = random.choices(np.arange(self.action_size), k=BATCH_SIZE)
      return random_action
  
  def learn(self, experiences, gamma):
    print('Started learning')
    states, actions, rewards, next_states, done = experiences
    criterion = torch.nn.BCELoss()
    self.qnet_local.train()
    self.qnet_target.eval()

    print('before predicting targets')

    #predicted_targets = self.qnet_local(states)#.gather(1, actions)

    #print('Predicted targets : {}'.format(predicted_targets))
    #print(next_states.view(1, 1))
    with torch.no_grad():
      labels_next = self.qnet_target(next_states).detach().max(1)[0].unsqueeze(1)
    
    print('Rewards : ', rewards)
    labels = 1 + (gamma * labels_next)
    predicted_targets = self.qnet_local(states).gather(1, actions.long())

    print('Predicted targets : ', predicted_targets.gather(1, actions.long()))
    print('Labels : ', labels)

    loss = criterion(predicted_targets, labels).to(device)
    print("Training loss :: {}".format(loss))
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    # perform soft update
    self.soft_update(self, self.qnet_local, self.qnet_target, TAU)
  
  def soft_update(self, local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)




In [0]:
class ReplayBuffer():
  def __init__(self, action_size, buffer_size, batch_size, seed):
    self.action_size = action_size
    self.memory = deque(maxlen=buffer_size)
    self.batch_size = batch_size
    self.experiences = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    self.seed = random.seed(seed)
  
  def add(self, state, action, reward, next_state, done):
    experience = self.experiences(state, action, reward, next_state, done)
    self.memory.append(experience)

  def sample(self):
    experiences = random.sample(self.memory, k=self.batch_size)
    states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).double().to(device)
    actions = torch.from_numpy(np.vstack([e.action.cpu() for e in experiences if e is not None])).double().to(device)
    rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).double().to(device)
    next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).double().to(device)
    dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None])).double().to(device)

    # experiences = random.sample(self.memory, k=BATCH_SIZE)

    # batch = self.experiences(*zip(experiences))

    # states = torch.cat(batch.state)
    # actions = torch.cat(batch.actions)
    # rewards = torch.cat(batch.reward)
    # next_states = torch.cat(batch.next_state)
    # dones = torch.cat(batch.done)

    return (states, actions, rewards, next_states, dones)
  
  def __len__(self):
      return len(self.memory)


# Training the agent

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

train = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

In [0]:
#df = pd.read_csv('./dataset/creditcard.csv')

In [230]:
# check reward strategy once
# add probability to epsilon_greedy
deep_agent = Agent(action_size=2, seed=0)
num_episodes = 200
max_t = 1000
state = 0

for i in range(num_episodes):  
  env.state_index = 0
  current_state = df.loc[0]
  next_state = df.iloc[1, :-1].values
  print(next_state)
  #print('Current state : ', current_state)
  score = 0
  for state_idx, data in enumerate(train_loader, 0):
    inputs, labels = data    
    action = deep_agent.epsilon_greedy_action(inputs)
    #print('action taken : ', action)
    next_stated, reward, done, _ = env.step(action, BATCH_SIZE)
    #print('Next_state : ', next_state)
    #print('Next state dtype ; ', next_state.dtype)
    #print(next_state)
    deep_agent.step(current_state, action, reward, next_state, done)
    current_state = next_state
    #state = next_state
    #score += reward      

[ 0.          1.19185711  0.26615071  0.16648011  0.44815408  0.06001765
 -0.08236081 -0.07880298  0.08510165 -0.25542513 -0.16697441  1.61272666
  1.06523531  0.48909502 -0.1437723   0.63555809  0.46391704 -0.11480466
 -0.18336127 -0.14578304 -0.06908314 -0.22577525 -0.63867195  0.10128802
 -0.33984648  0.1671704   0.12589453 -0.0089831   0.01472417  2.69      ]
Experience sampled from memory :  (tensor([[ 0.0000,  1.1919,  0.2662,  0.1665,  0.4482,  0.0600, -0.0824, -0.0788,
          0.0851, -0.2554, -0.1670,  1.6127,  1.0652,  0.4891, -0.1438,  0.6356,
          0.4639, -0.1148, -0.1834, -0.1458, -0.0691, -0.2258, -0.6387,  0.1013,
         -0.3398,  0.1672,  0.1259, -0.0090,  0.0147,  2.6900]],
       device='cuda:0', dtype=torch.float64), tensor([[1.]], device='cuda:0', dtype=torch.float64), tensor([], device='cuda:0', size=(1, 0), dtype=torch.float64), tensor([[ 0.0000,  1.1919,  0.2662,  0.1665,  0.4482,  0.0600, -0.0824, -0.0788,
          0.0851, -0.2554, -0.1670,  1.6127,  1

TypeError: ignored

In [0]:
credit_card_dataset = './dataset/creditcard.csv'
df_credit_card = pd.DataFrame(pd.read_csv(credit_card_dataset))

In [167]:
print(df_credit_card.iloc[0, :])

Time        0.000000
V1         -1.359807
V2         -0.072781
V3          2.536347
V4          1.378155
V5         -0.338321
V6          0.462388
V7          0.239599
V8          0.098698
V9          0.363787
V10         0.090794
V11        -0.551600
V12        -0.617801
V13        -0.991390
V14        -0.311169
V15         1.468177
V16        -0.470401
V17         0.207971
V18         0.025791
V19         0.403993
V20         0.251412
V21        -0.018307
V22         0.277838
V23        -0.110474
V24         0.066928
V25         0.128539
V26        -0.189115
V27         0.133558
V28        -0.021053
Amount    149.620000
Class       0.000000
Name: 0, dtype: float64
